In [1]:
import os
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader



In [2]:
#create custom data class

class CustomDataset(Dataset):
    def __init__(self,low_idx,high_idx):
        self.data = pd.read_csv("./dataset/IRIS.csv", sep=",", skiprows=range(low_idx,high_idx), header=1)
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sepal_length = torch.tensor(self.data.iloc[idx, 0])
        sepal_width = self.data.iloc[idx, 1]
        petal_length = self.data.iloc[idx, 2]
        petal_width = self.data.iloc[idx, 3] 
        label = self.data.iloc[idx, 4]
        
        tensor_data = torch.tensor([sepal_length,sepal_width,sepal_width,sepal_width],dtype=torch.float32)
        
        return tensor_data , label

In [3]:
#just my afwull way to seperate training and testing dataset
# 80% training , 20% Test

train_dataset = CustomDataset(120,149)
test_dataset = CustomDataset(1,120)


In [4]:
#create train_dataloader and test_dataloader
# 80% for training, 20% of data for testing -> 149*0.8 = 119.2 -> 120 , 29
train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True) 
test_loader = DataLoader(test_dataset, batch_size=10, shuffle=True)

In [5]:
#4 inputs and 3  outputs 
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_stack =  nn.Sequential(
         nn.Linear(4,128),
         nn.ReLU(),
         nn.Linear(128,64),
         nn.ReLU(),
         nn.Linear(64,3),
         )
    def forward(self, x):
        logits = self.linear_stack(x)
        return logits

In [6]:
#map labels into tensor of shape [1, 0, 0]
def transform_label(label_data):
    data = []
    for i in label_data:
        if i == "Iris-setosa":
            data.append(torch.tensor([1,0,0], dtype=torch.float32))
        elif i == "Iris-versicolor": 
            data.append(torch.tensor([0,1,0], dtype=torch.float32))
        else:
            data.append(torch.tensor([0,0,1], dtype=torch.float32))
    return torch.stack(data)

In [7]:
#train_labels, X_train = next(iter(train_loader))
X_train, train_labels = next(iter(train_loader))
train_labels = transform_label(train_labels)
X_train.shape, train_labels.shape, X_train.dtype, train_labels.dtype

(torch.Size([10, 4]), torch.Size([10, 3]), torch.float32, torch.float32)

In [8]:
X_train

tensor([[6.3000, 2.5000, 2.5000, 2.5000],
        [5.9000, 3.0000, 3.0000, 3.0000],
        [6.5000, 3.2000, 3.2000, 3.2000],
        [7.7000, 2.6000, 2.6000, 2.6000],
        [5.1000, 3.7000, 3.7000, 3.7000],
        [5.6000, 2.5000, 2.5000, 2.5000],
        [6.7000, 3.1000, 3.1000, 3.1000],
        [5.8000, 2.8000, 2.8000, 2.8000],
        [5.2000, 3.4000, 3.4000, 3.4000],
        [7.6000, 3.0000, 3.0000, 3.0000]])

In [9]:
train_labels

tensor([[0., 1., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        [1., 0., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [1., 0., 0.],
        [0., 0., 1.]])

In [10]:
#create model
lr = 0.01
model = NeuralNetwork()
optim = torch.optim.Adam(model.parameters(), lr=lr)
loss = torch.nn.CrossEntropyLoss()
#Weights are by default torch.32 not 64 --> error message 

In [11]:
n_iters = 12
for epochs in range(n_iters):
    #forward pass and loss
    #test_labels, X_test = next(iter(test_loader))
    print(f"\n epochs: {epochs+1}/{n_iters} \n")
    for i,(inputs, labels) in enumerate(train_loader):
        out = model(inputs)
        train_labels = transform_label(labels)
        l = loss(out, train_labels)
        #backward pass
        l.backward()
        #update weights
        optim.step()
        optim.zero_grad()
    
        if i % 1 == 0:
            print(f" steps: {i+1}, loss : {l.item()}")


 epochs: 1/12 

 steps: 1, loss : 1.3056352138519287
 steps: 2, loss : 2.1316020488739014
 steps: 3, loss : 1.046773910522461
 steps: 4, loss : 0.9722232818603516
 steps: 5, loss : 0.8581293225288391
 steps: 6, loss : 0.8325742483139038
 steps: 7, loss : 0.8303266763687134
 steps: 8, loss : 0.9274429082870483
 steps: 9, loss : 1.0914428234100342
 steps: 10, loss : 1.0757962465286255
 steps: 11, loss : 0.8785656690597534
 steps: 12, loss : 0.9055712819099426

 epochs: 2/12 

 steps: 1, loss : 0.7583292126655579
 steps: 2, loss : 0.9304589033126831
 steps: 3, loss : 1.0484877824783325
 steps: 4, loss : 0.7265259027481079
 steps: 5, loss : 0.8892237544059753
 steps: 6, loss : 0.790526807308197
 steps: 7, loss : 0.5628173351287842
 steps: 8, loss : 0.7612552046775818
 steps: 9, loss : 0.9337688684463501
 steps: 10, loss : 0.7428701519966125
 steps: 11, loss : 0.49663275480270386
 steps: 12, loss : 0.5852622985839844

 epochs: 3/12 

 steps: 1, loss : 0.6489673852920532
 steps: 2, loss : 0

In [12]:
input_test = torch.tensor([4.9,3.1,1.5,0.1])

In [13]:
model(input_test)

tensor([-4.4558,  2.2277,  1.5091], grad_fn=<AddBackward0>)

In [14]:
test_loader = DataLoader(test_dataset, batch_size=10, shuffle=True)
X_test, test_labels = next(iter(train_loader))
test_labels = transform_label(test_labels)

In [15]:
x_label_predicted = model(X_test)

In [16]:
x_label_predicted

tensor([[-2.7990,  1.4190,  0.7639],
        [-4.8114,  2.4504,  1.7064],
        [ 3.0509, -1.3122, -2.6861],
        [ 6.7025, -2.7783, -5.0206],
        [-4.1604,  2.1224,  1.4235],
        [ 4.2213, -1.7859, -3.3817],
        [-5.8072,  2.9393,  2.1220],
        [ 2.0417, -0.8851, -2.0907],
        [-4.8876,  2.4791,  1.7026],
        [ 4.3303, -1.8431, -3.4737]], grad_fn=<AddmmBackward0>)

In [17]:
_, x_label_predicted_hat = torch.max(x_label_predicted,1)

In [18]:
x_label_predicted_hat

tensor([1, 1, 0, 0, 1, 0, 1, 0, 1, 0])

In [19]:
def final_x(x_hat):
    list_tensors = []
    for i in x_hat:
        final_tensor = torch.zeros(1,3)
        final_tensor[0:,i] = 1
        list_tensors.append(final_tensor)
    return torch.cat(list_tensors)
    
    # tensor[0:,1] = 4 to accses specific value 

In [20]:
final_x = final_x(x_label_predicted_hat)

In [21]:
#final_x, test_labels
#print(f"{})

In [22]:
def get_accuracy(model_prediction, test_labels):
    idx = 0
    number_pred = 0
    counter = 0
    while idx < 10:
        print(f" \n Tensor[{idx+1}]")
        for i in range(3):
            print(f"model: {model_prediction[idx][i].item()}, label: {test_labels[idx][i].item()}")
            if model_prediction[idx][i].item() == test_labels[idx][i].item():

                counter += 1
                if counter == 3:
                    number_pred += 1
        counter = 0
          
        idx +=1
    accuracy = number_pred/10 *100
    return print(f"\naccuracy of model {number_pred/10 *100}%")

In [23]:
get_accuracy(final_x, test_labels)

 
 Tensor[1]
model: 0.0, label: 0.0
model: 1.0, label: 1.0
model: 0.0, label: 0.0
 
 Tensor[2]
model: 0.0, label: 0.0
model: 1.0, label: 1.0
model: 0.0, label: 0.0
 
 Tensor[3]
model: 1.0, label: 1.0
model: 0.0, label: 0.0
model: 0.0, label: 0.0
 
 Tensor[4]
model: 1.0, label: 1.0
model: 0.0, label: 0.0
model: 0.0, label: 0.0
 
 Tensor[5]
model: 0.0, label: 0.0
model: 1.0, label: 1.0
model: 0.0, label: 0.0
 
 Tensor[6]
model: 1.0, label: 1.0
model: 0.0, label: 0.0
model: 0.0, label: 0.0
 
 Tensor[7]
model: 0.0, label: 0.0
model: 1.0, label: 1.0
model: 0.0, label: 0.0
 
 Tensor[8]
model: 1.0, label: 1.0
model: 0.0, label: 0.0
model: 0.0, label: 0.0
 
 Tensor[9]
model: 0.0, label: 0.0
model: 1.0, label: 0.0
model: 0.0, label: 1.0
 
 Tensor[10]
model: 1.0, label: 1.0
model: 0.0, label: 0.0
model: 0.0, label: 0.0

accuracy of model 90.0%
